**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

See [Configure Service Account](https://kubernetes.io/docs/tasks/configure-pod-container/configure-service-account/)

In [2]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  creationTimestamp: "2022-01-20T15:16:42Z"
  name: default
  namespace: default
  resourceVersion: "425"
  uid: 2b5f0b66-a485-40d5-a964-82756a140421
secrets:
- name: default-token-zsxxb


In [3]:
kubectl patch serviceaccount default  --patch "automountServiceAccountToken: false"

serviceaccount/default patched


In [4]:
kubectl get serviceaccount default -o yaml | grep -E '(^|automountServiceAccountToken.*)'

apiVersion: v1
automountServiceAccountToken: false
kind: ServiceAccount
metadata:
  creationTimestamp: "2022-01-20T15:16:42Z"
  name: default
  namespace: default
  resourceVersion: "1072"
  uid: 2b5f0b66-a485-40d5-a964-82756a140421
secrets:
- name: default-token-zsxxb


# ☠️ **this can be overwritten** ☠️



In [14]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-with-token-again
  labels:
    app: nginx-with-token-again
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx-with-token-again
  template:
    metadata:
      labels:
        app: nginx-with-token-again
    spec:
      automountServiceAccountToken: true
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps/nginx-with-token-again configured


In [10]:
kubectl get pods
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' )
echo $POD

NAME                                     READY   STATUS    RESTARTS   AGE
nginx-with-token-again-7d7cd958c-szlwq   1/1     Running   0          12s
nginx-with-token-again-7d7cd958c-szlwq


In [11]:
kubectl exec  $POD -- df

Filesystem             1K-blocks      Used Available Use% Mounted on
overlay                308587328 234169208  58673096  80% /
tmpfs                      65536         0     65536   0% /dev
tmpfs                   32632360         0  32632360   0% /sys/fs/cgroup
/dev/mapper/system-var 308587328 234169208  58673096  80% /etc/hosts
shm                        65536         0     65536   0% /dev/shm
tmpfs                   32632360         0  32632360   0% /proc/acpi
tmpfs                   32632360         0  32632360   0% /proc/scsi
tmpfs                   32632360         0  32632360   0% /sys/firmware


In [12]:
kubectl get pods  -o yaml | grep -i token

    generateName: nginx-with-token-again-7d7cd958c-
      app: nginx-with-token-again
    name: nginx-with-token-again-7d7cd958c-szlwq
      name: nginx-with-token-again-7d7cd958c


In [15]:
kubectl get pods -A -o jsonpath='{.items[*].spec.automountServiceAccountToken}'

true